In [1]:
import os

# 让 GDAL/rasterio 不输出那些会触发编码问题的日志
os.environ["CPL_DEBUG"] = "OFF"
os.environ["CPL_LOG"] = "NUL"              # Windows 的空设备（等价于 /dev/null）
os.environ["GDAL_DISABLE_READDIR_ON_OPEN"] = "EMPTY_DIR"

# 可选：减少并发，避免多线程里更难定位
os.environ["GDAL_NUM_THREADS"] = "1"
os.environ["VSI_CACHE"] = "TRUE"
os.environ["VSI_CACHE_SIZE"] = "5000000"


In [2]:
import os, sys
from pathlib import Path

proj_dir = Path(sys.prefix) / "Library" / "share" / "proj"
os.environ["PROJ_DATA"] = str(proj_dir)
os.environ["PROJ_LIB"]  = str(proj_dir)

import pyproj
from pyproj import datadir
datadir.set_data_dir(str(proj_dir))

from pyproj import CRS
print("CRS 4326:", CRS.from_epsg(4326))


CRS 4326: EPSG:4326


In [3]:
from pystac_client import Client
import planetary_computer as pc
import stackstac
import numpy as np
import matplotlib.pyplot as plt


In [4]:
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

bbox = (139.98, 35.86, 140.02, 35.90)

search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox,
    datetime="2025-10-01/2025-12-17",
    query={"eo:cloud_cover": {"lt": 60}},
)
items = list(search.get_items())
item = sorted(items, key=lambda i: i.properties.get("eo:cloud_cover", 9999))[0]
signed_item = pc.sign(item)

epsg = 32654

da_test = stackstac.stack(
    [signed_item],
    assets=["B04"],         # 只读一个 band
    bounds_latlon=bbox,
    epsg=epsg,
    resolution=60,          # 更粗更快
).isel(time=0)

da_test


C:\Users\laval\miniforge3\envs\pc\Lib\site-packages\pystac_client\item_search.py:925: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


<xarray.DataArray 'stackstac-a8e48089df36406f498afa37058050ce' (band: 1, y: 76,
                                                                x: 62)> Size: 38kB
dask.array<getitem, shape=(1, 76, 62), dtype=float64, chunksize=(1, 76, 62), chunktype=numpy.ndarray>
Coordinates: (12/45)
  * band                                     (band) <U3 12B 'B04'
  * y                                        (y) float64 608B 3.973e+06 ... 3...
  * x                                        (x) float64 496B 4.079e+05 ... 4...
    time                                     datetime64[ns] 8B 2025-12-07T01:...
    id                                       <U54 216B 'S2B_MSIL2A_20251207T0...
    s2:datastrip_id                          <U64 256B 'S2B_OPER_MSI_L2A_DS_2...
    ...                                       ...
    proj:bbox                                object 8B {399960.0, 509760.0, 3...
    title                                    <U18 72B 'Band 4 - Red - 10m'
    common_name                              <U3 12B 'red'
    center_wavelength                        float64 8B 0.665
    full_width_half_max                      float64 8B 0.038
    epsg                                     int64 8B 32654
Attributes:
    spec:        RasterSpec(epsg=32654, bounds=(407880, 3968820, 411600, 3973...
    crs:         epsg:32654
    transform:   | 60.00, 0.00, 407880.00|\n| 0.00,-60.00, 3973380.00|\n| 0.0...
    resolution:  60

In [5]:
arr = da_test.isel(band=0).values
arr.shape, float(np.nanmin(arr)), float(np.nanmax(arr))


SystemError: <class 'UnicodeDecodeError'> returned a result with an exception set